In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.6 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from agents import Agent, Runner ,OpenAIChatCompletionsModel,RunConfig, AsyncOpenAI,  ModelSettings,  function_tool, set_default_openai_client, set_default_openai_api
from agents.agent import StopAtTools
from google.colab import userdata

gemini_api_key = userdata.get("GOOGLE_API_KEY")

set_default_openai_api("chat_completions")

if not gemini_api_key:
  raise ValueError("Gemini Key is not set. Pleaes set it first")


@function_tool(name_override="Addition_function" , description_override="Add two numbers which have been given to you") # Name will be Addition_function and same as description. It overrides.
def add(a:int, b:int)->int:
  """ Addition of two numbers """
  print(" \n [DEBUG] Addition tool called")
  return a + b - 5

@function_tool
def human_loop() -> str:
  """ Human in the loop """
  print("Human in the loop")
  return

client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    openai_client=client,
    model='gemini-2.0-flash'
)


config = RunConfig(
    model=model,
    model_provider=client,
    tracing_disabled=True
)


agent: Agent = Agent(
  name="Agent",
  instructions="You are an Agent",
  model=model,
  tools=[add, human_loop],
  # tool_use_behavior=StopAtTools(stop_at_tool_names=['human_loop'])
  # model_settings=ModelSettings(tool_choice="required")
  # reset_tool_choice=False
)


In [6]:
res = Runner.run_sync(
    agent , "Hi, what is 2 + 2",
    # max_turns=2
)

print(res.final_output)

 
 [DEBUG] Addition tool called
I'm not sure I got that right. Would you like me to get a human to check?



In [4]:
from agents import Agent , Runner , RunConfig, function_tool, AsyncOpenAI, RunContextWrapper, FunctionTool
from pydantic import BaseModel, ConfigDict
from typing import Any


class FunctionArgs(BaseModel):
  username: str
  age: int
  model_config = ConfigDict(extra="forbid")


async def run_function(ctx: RunContextWrapper[Any], args: str) -> str
  parsed = FunctionArgs.model_validate_json(args)
  return


tool = FunctionTool(
    name='Process_User',
    description='Process Extracted user data',
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function
)

example_agent: Agent = Agent(
  name="Example Agent",
  instructions="You are an Agent",
  tools=[tool]
)
